In [ ]:
!pip install openai

In [ ]:
!pip install requests

In [ ]:
import openai
import json
import os

# Replace 'your_api_key_here' with your actual API key
openai.api_key = 'your_api_key_here'

headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}",
    }

# This is the main GPT prompt.  This instruction will be combined with the text of the chat transcript.  This prompt should include the entire definition of the code scale, as well as examples of difficult issues and how to address them. It must also address the format of the desired response.
# Example prompt:
prompt1 = """
Your response must be in this format and order:
DESIRED FORMAT

You are classifying a library chat transcript based on the following scale to score library chat interactions: EXAMPLE READ SCALE
Score 0 - Not a question; Prank; Error
Score 1 - No specialized knowledge, no resource consultation (includes canned resource links).  Examples:  Directional questions for locations within the library; Any question answered without resource consultation; Library Hours; Public access point phone numbers
Score 2 - Minimal skills, nominal resource consultation.  Examples:  Call number directions; Library policy; Known item search; Individual staff contact information
Score 3 - Some reference knowledge required and/or some effort required.  Examples: Topic search in single database; Basic instruction on searching; finding subject databases; complex technical tasks (e.g. how to save PDFs or export citations)
Score 4 - Advanced reference, multiple sources, subject knowledge useful but not required.  Examples:  Lengthy topic searches for multiple document types;  Multiple topic searches across databases
Score 5 - Subject knowledge required, lengthy consultation, multiple resources.
Score 6 - Research effort beyond patron interaction.  Should never happen in chat reference.
Score Unclear - not enough information in the transcript to make a clear decision

Further instructions and clarifications:
ADDITIONAL INSTRUCTIONS

Transcript (starting with initial question):

"""



In [ ]:
import time

#In the Springshare LibChat export, the initial question and the chat transcript are separated into 2 columns.
#We need to create a text file with the initial question + chat transcript combined.  Each transcript should be on a single line.
#Additional data cleaning may be helpful, but does not seem to be strictly necessary.
#Input file should be in UTF-16 encoding

input_file = "input_file.txt"

# Read the file as binary and decode it, ignoring NUL characters
with open(input_file, "rb") as file:
    content = file.read().decode("utf-16", errors="ignore")

# Convert the decoded content into a list of lines
lines = content.splitlines()


In [ ]:
def send_text_and_save_response(prompt, line_number):
    
    
    # Call the GPT-4 API
    response = openai.Completion.create(**params)
    
    # Extract the response text
    response_text = response.choices[0].text.strip()
    
    # Save the response to a new file
    output_file_name = f"gpt3_response_{line_number}.txt"
    with open(output_file_name, "w") as output_file:
        output_file.write(response_text)
    
    print(f"Response saved to '{output_file_name}'")

In [ ]:
def send_text_and_save_response(prompt, line_number):
    # Set your OpenAI API key
    api_key = "your_api_key_here"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
    }

    # Set the parameters for the GPT-4 API call
    # gpt-4 model is costlier, but has been shown to increase accuracy from 75% to 96%
    # Other parameters will affect the output but have not been systematically tested
    data = {
        "model": "gpt-4",
        "messages": [{"role": "system", "content": "You are a helpful assistant."},
                     {"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 400,
        "n": 1,
        "stop": None,
    }
    
    # Call the GPT-4 API with the chat-completions endpoint
    # A timer is set to reduce the number of errors due to timeout
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))
    
    if response.status_code == 429:
        time.sleep(2)
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))
        
        if response.status_code == 429:
            time.sleep(30)
            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))
            
            if response.status_code == 200:
                response_data = response.json()
                print("Response data:", response_data)
                # Extract the response text
                response_text = response_data["choices"][0]["message"]["content"].strip()

                # Save the response to a new file
                output_file_name = f"gpt3_response.txt"
                with open(output_file_name, "a", encoding="utf-8") as output_file:
                    output_file.write(str(line_number) + ";" + response_text + "\n")
                    print(f"Response saved to '{output_file_name}'")
    
        
            else:
                print(f"Error: Request failed with status code {response.status_code}")
                print(f"Response content: {response.content}")
            
            
            
        if response.status_code == 200:
            response_data = response.json()
            print("Response data:", response_data)
            # Extract the response text
            response_text = response_data["choices"][0]["message"]["content"].strip()

            # Save the response to a new file
            output_file_name = f"gpt3_response.txt"
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(str(line_number) + ";" + response_text + "\n")
                print(f"Response saved to '{output_file_name}'")
    
        
        else:
            print(f"Error: Request failed with status code {response.status_code}")
            print(f"Response content: {response.content}")
        
        
    else:
        
        # Check if the request was successful
        if response.status_code == 200:
            response_data = response.json()
            print("Response data:", response_data)
            # Extract the response text
            response_text = response_data["choices"][0]["message"]["content"].strip()

            # Save the response to a new file
            output_file_name = f"gpt3_response.txt"
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(str(line_number) + ";" + response_text + "\n")
                print(f"Response saved to '{output_file_name}'")
    
        
        else:
            print(f"Error: Request failed with status code {response.status_code}")
            print(f"Response content: {response.content}")

In [ ]:
import csv
import requests
# Read the input file
with open(input_file, "r", encoding="utf-16") as file:
    file_reader = csv.reader(file, delimiter='\t')
    index = 0
    for line in file_reader:
        print(f"Processing line {index}")
        print(line)

        # Send the text and save the response
        send_text_and_save_response(prompt1 + " " + line[0], index)

        index += 1